##### 0 - Requirementes and configurations

In [99]:
# Set autoreload
%load_ext autoreload
%autoreload 2

# Python modules
import logging
import pandas as pd

# Data modules
from src.data_handler.crypto_price_fetcher import get_crypto_data
from src.data_handler.news_processor import process_news


# Set logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)
logger = logging.getLogger(__name__)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [119]:
# Get Prices
btc_data = get_crypto_data('BTC', 'USDT', 'hour', day=1, month=1, year=2019)

# Get News
news = pd.read_csv('data/news/raw/news_btc.csv', nrows=sample)

2025-03-10 03:14:39,042 - INFO - Fetching BTC/USDT hourly data from 2019-01-01...
2025-03-10 03:14:39,872 - INFO - Successfully fetched 2001 records
2025-03-10 03:14:39,887 - INFO - Data saved to data/prices/BTC_USDT_hour_2019-01-01_2260days.csv


In [139]:
# Build events

#% Proccess news
import pandas as pd
sample = 1000 # first 1000 starting in march 2019
events = process_news(sample=sample)
events['date'] = pd.to_datetime(events['date'])
events = events.sort_values(by='date')

# Drop events with nan in text column
events = events.dropna(subset=['text'])

#% Join events with stock data based on date use closes time reference
events = pd.merge(events, btc_data, how='left', left_on='date', right_on='timestamp')

# Create a new column with the next day's close price
events['next_t_close'] = events['close'].shift(-100)

# Calculate close price rolling stats 7 ts, 30 ts, 90 ts
events['close_7'] = events['close'].rolling(7).mean()
events['close_30'] = events['close'].rolling(30).mean()
events['close_90'] = events['close'].rolling(90).mean()

# Drop first 100 rows
events = events.dropna(subset=['next_t_close'])
events = events.dropna(subset=['close_90'])

# if next_day_close is lower than close, then 1 else 0
events['target'] = events.apply(lambda x: "Long" if x['next_t_close'] > x['close'] else "short", axis=1)

# calculate the difference between next_day_close and close
events['diff_perc'] = ((events['next_t_close'] / events['close']) - 1) * 100

Processed 1000 news articles


<string>:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.


In [ ]:
# Installing required packages (if not already installed)
# !pip install langchain langchain-openai pydantic pandas matplotlib

import os
import json
import datetime
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Dict, Any, Optional
from pydantic import BaseModel, Field

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.schema import HumanMessage, AIMessage

# Set your API key for OpenAI
os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# --- Data Structures ---

class NewsItem(BaseModel):
    """A news item with its features."""
    content: str
    date: str
    source: str
    category: str
    sentiment: Optional[float] = 0.0

class PriceData(BaseModel):
    """Price data point with extended fields."""
    asset: str
    price: float
    date: str
    open: Optional[float] = None
    high: Optional[float] = None
    low: Optional[float] = None
    volume: Optional[float] = None
    next_t_close: Optional[float] = None
    close_7: Optional[float] = None
    close_30: Optional[float] = None
    close_90: Optional[float] = None
    target: Optional[str] = None
    diff_perc: Optional[float] = None

class Decision(BaseModel):
    """A decision made by the agent."""
    decision_id: str = Field(default_factory=lambda: f"decision_{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}")
    recommendation: str
    confidence: float
    reasoning: str
    timestamp: str = Field(default_factory=lambda: datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    outcome: Optional[str] = None
    reward: Optional[float] = None

class Fact(BaseModel):
    """A long term fact the agent has learned."""
    fact: str
    source: str
    confidence: float
    category: str
    timestamp: str = Field(default_factory=lambda: datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

# --- Memory Components ---

class SensoryMemory:
    def __init__(self):
        self.news: List[NewsItem] = []
        self.prices: List[PriceData] = []
        self.max_items = 10  # Keep only the most recent items
        
    def add_news(self, news_item: NewsItem):
        self.news.append(news_item)
        if len(self.news) > self.max_items:
            self.news.pop(0)
            
    def add_price(self, price_data: PriceData):
        self.prices.append(price_data)
        if len(self.prices) > self.max_items:
            self.prices.pop(0)
            
    def get_formatted(self) -> str:
        news_str = "\n".join([f"- {item.date}: {item.content} (Source: {item.source}, Category: {item.category})" 
                             for item in self.news])
        price_str = "\n".join([f"- {item.date}: {item.asset} close=${item.price}" 
                              for item in self.prices])
        return f"## CURRENT SENSORY INPUT\n### Recent News:\n{news_str}\n\n### Current Prices:\n{price_str}"

class ShortTermMemory:
    def __init__(self):
        self.news: List[NewsItem] = []
        self.prices: Dict[str, List[PriceData]] = {}  # Keyed by asset
        self.max_news = 15
        
    def add_news(self, news_item: NewsItem):
        self.news.append(news_item)
        if len(self.news) > self.max_news:
            self.news.pop(0)
            
    def add_price(self, price_data: PriceData):
        if price_data.asset not in self.prices:
            self.prices[price_data.asset] = []
        self.prices[price_data.asset].append(price_data)
        if len(self.prices[price_data.asset]) > 30:
            self.prices[price_data.asset].pop(0)
            
    def get_price_averages(self) -> Dict[str, float]:
        averages = {}
        for asset, price_list in self.prices.items():
            if price_list:
                averages[asset] = sum(p.price for p in price_list) / len(price_list)
        return averages
    
    def get_formatted(self) -> str:
        news_str = "\n".join([f"- {item.date}: {item.content}" for item in self.news])
        averages = self.get_price_averages()
        price_summary = "\n".join([f"- {asset}: Average=${avg:.2f}, Latest=${self.prices[asset][-1].price:.2f}"
                                  for asset, avg in averages.items()])
        return f"## SHORT TERM MEMORY\n### Recent Historical News (Past few days):\n{news_str}\n\n### Price Trends:\n{price_summary}"

class ProceduralMemory:
    def __init__(self):
        self.procedures: Dict[str, str] = {
            "news_analysis": """
To analyze news items:
1. Identify key entities (companies, markets, sectors)
2. Assess sentiment (positive, negative, neutral)
3. Categorize impact (short-term, long-term, speculative)
4. Cross-reference with other news for confirmation
5. Evaluate source reliability
            """,
            "price_analysis": """
To analyze price movements:
1. Calculate percentage changes (daily, weekly)
2. Identify trends (upward, downward, sideways)
3. Compare to historical averages
4. Check for correlation with news events
5. Note unusual volume or volatility
            """,
            "decision_making": """
To make recommendation decisions:
1. Assess all available news and price data
2. Consider short-term and long-term implications
3. Evaluate risk factors
4. Form a hypothesis backed by specific data points
5. Assign a confidence level based on data quality and consistency
6. Make a clear recommendation with supporting rationale
            """
        }
        
    def add_procedure(self, name: str, procedure: str):
        self.procedures[name] = procedure
        
    def get_procedure(self, name: str) -> str:
        return self.procedures.get(name, "Procedure not found.")
    
    def get_formatted(self) -> str:
        procedures_str = "\n\n".join([f"### {name.upper()}:\n{proc}" 
                                     for name, proc in self.procedures.items()])
        return f"## PROCEDURAL MEMORY - HOW TO ANALYZE AND DECIDE\n{procedures_str}"

class LongTermMemory:
    def __init__(self):
        self.facts: List[Fact] = []
        
    def add_fact(self, fact: Fact):
        self.facts.append(fact)
        
    def get_facts_by_category(self, category: str) -> List[Fact]:
        return [f for f in self.facts if f.category == category]
    
    def get_formatted(self, max_facts: int = 15) -> str:
        recent_facts = sorted(self.facts, key=lambda x: x.timestamp, reverse=True)[:max_facts]
        facts_str = "\n".join([f"- {fact.fact} (Confidence: {fact.confidence}, Source: {fact.source})" 
                              for fact in recent_facts])
        return f"## LONG TERM MEMORY - ESTABLISHED FACTS\n{facts_str}"

class AutobiographicalMemory:
    def __init__(self):
        self.decisions: List[Decision] = []
        
    def add_decision(self, decision: Decision):
        self.decisions.append(decision)
        
    def update_outcome(self, decision_id: str, outcome: str, reward: float):
        for decision in self.decisions:
            if decision.decision_id == decision_id:
                decision.outcome = outcome
                decision.reward = reward
                break
                
    def get_formatted(self, max_decisions: int = 10) -> str:
        recent_decisions = sorted(self.decisions, key=lambda x: x.timestamp, reverse=True)[:max_decisions]
        decisions_str = "\n".join([
            f"- Decision {d.decision_id}: {d.recommendation} (Confidence: {d.confidence})\n  "
            f"Reasoning: {d.reasoning}\n  Outcome: {d.outcome or 'Pending'}, Reward: {d.reward or 'N/A'}"
            for d in recent_decisions
        ])
        return f"## AUTOBIOGRAPHICAL MEMORY - PREVIOUS DECISIONS AND OUTCOMES\n{decisions_str}"

class WorkingMemory:
    def __init__(self):
        self.thought_process: List[str] = []
        self.max_thoughts = 5
        
    def add_thought(self, thought: str):
        self.thought_process.append(thought)
        if len(self.thought_process) > self.max_thoughts:
            self.thought_process.pop(0)
            
    def clear(self):
        self.thought_process = []
        
    def get_formatted(self) -> str:
        thoughts_str = "\n".join([f"{i+1}. {thought}" for i, thought in enumerate(self.thought_process)])
        return f"## WORKING MEMORY - CURRENT REASONING PROCESS\n{thoughts_str}"

class ProspectiveMemory:
    def __init__(self):
        self.considerations: List[str] = []
        
    def add_consideration(self, consideration: str):
        self.considerations.append(consideration)
        
    def get_formatted(self) -> str:
        if not self.considerations:
            return "## PROSPECTIVE MEMORY - FUTURE CONSIDERATIONS\nNo specific future considerations at this time."
        considerations_str = "\n".join([f"- {consideration}" for consideration in self.considerations])
        return f"## PROSPECTIVE MEMORY - FUTURE CONSIDERATIONS\n{considerations_str}"

# --- The main LLM Agent ---

class LLMAgent:
    def __init__(self, model_name="gpt-3.5-turbo"):
        self.sensory_memory = SensoryMemory()
        self.short_term_memory = ShortTermMemory()
        self.procedural_memory = ProceduralMemory()
        self.long_term_memory = LongTermMemory()
        self.autobiographical_memory = AutobiographicalMemory()
        self.working_memory = WorkingMemory()
        self.prospective_memory = ProspectiveMemory()
        self.llm = ChatOpenAI(model_name=model_name, temperature=0.7)
        self.system_prompt = (
            "You are an advanced AI financial analyst with multiple memory systems.\n"
            "Your goal is to provide investment recommendations based on news and price data.\n"
            "As you process information, you will build knowledge and learn from your past decisions.\n"
            "Think step by step and show your reasoning process before making final recommendations.\n"
        )

    def _build_full_prompt(self, query: str) -> str:
        prompt_parts = [
            self.sensory_memory.get_formatted(),
            self.short_term_memory.get_formatted(),
            self.procedural_memory.get_formatted(),
            self.long_term_memory.get_formatted(),
            self.autobiographical_memory.get_formatted(),
            self.working_memory.get_formatted(),
            self.prospective_memory.get_formatted(),
            f"\n## NEW QUERY\n{query}\n\n## RESPONSE\nLet me think through this step by step:"
        ]
        return "\n\n".join(prompt_parts)
    
    def react_step(self, query: str) -> str:
        full_prompt = self._build_full_prompt(query)
        response = self.llm.invoke([HumanMessage(content=full_prompt)])
        self.working_memory.add_thought(response.content)
        return response.content
    
    def make_recommendation(self, query: str) -> Decision:
        self.working_memory.clear()
        self.react_step(f"Analyze the latest news and price data to answer: {query}")
        recommendation_prompt = f"""
Based on your previous analysis, make a final recommendation regarding: {query}

Your response should be structured as:
RECOMMENDATION: [Clear statement of recommendation]
CONFIDENCE: [Numeric value between 0-1]
REASONING: [Concise summary of key factors that led to this recommendation]
"""
        full_prompt = self._build_full_prompt(recommendation_prompt)
        recommendation_response = self.llm.invoke([HumanMessage(content=full_prompt)])
        response_text = recommendation_response.content
        
        recommendation = ""
        confidence = 0.5  # Default
        reasoning = ""
        for line in response_text.split('\n'):
            if line.startswith("RECOMMENDATION:"):
                recommendation = line.replace("RECOMMENDATION:", "").strip()
            elif line.startswith("CONFIDENCE:"):
                confidence_str = line.replace("CONFIDENCE:", "").strip()
                try:
                    confidence = float(confidence_str)
                except ValueError:
                    if "high" in confidence_str.lower():
                        confidence = 0.8
                    elif "medium" in confidence_str.lower():
                        confidence = 0.5
                    elif "low" in confidence_str.lower():
                        confidence = 0.3
            elif line.startswith("REASONING:"):
                reasoning = line.replace("REASONING:", "").strip()
                
        decision = Decision(
            recommendation=recommendation,
            confidence=confidence,
            reasoning=reasoning
        )
        self.autobiographical_memory.add_decision(decision)
        return decision
    
    def process_feedback(self, decision_id: str, outcome: str, reward: float):
        self.autobiographical_memory.update_outcome(decision_id, outcome, reward)
        feedback_prompt = f"""
I received feedback on my recommendation (ID: {decision_id}):
Outcome: {outcome}
Reward: {reward}

Based on this feedback, what's one important fact I should remember for future decisions?
Format your response as:
NEW FACT: [concise statement of a fact to remember]
CATEGORY: [category for this fact]
CONFIDENCE: [numeric value between 0-1]
"""
        full_prompt = self._build_full_prompt(feedback_prompt)
        learning_response = self.llm.invoke([HumanMessage(content=full_prompt)])
        response_text = learning_response.content
        
        fact_text = ""
        category = "general"
        confidence = 0.5
        for line in response_text.split('\n'):
            if line.startswith("NEW FACT:"):
                fact_text = line.replace("NEW FACT:", "").strip()
            elif line.startswith("CATEGORY:"):
                category = line.replace("CATEGORY:", "").strip()
            elif line.startswith("CONFIDENCE:"):
                try:
                    confidence = float(line.replace("CONFIDENCE:", "").strip())
                except ValueError:
                    pass
        
        if fact_text:
            new_fact = Fact(
                fact=fact_text,
                source=f"Feedback on decision {decision_id}",
                confidence=confidence,
                category=category
            )
            self.long_term_memory.add_fact(new_fact)
            self.prospective_memory.add_consideration(
                f"Consider the outcome of similar situations to decision {decision_id} ({fact_text})"
            )
        return learning_response.content
    
    def update_with_news(self, news_items: List[NewsItem]):
        for item in news_items:
            self.sensory_memory.add_news(item)
            self.short_term_memory.add_news(item)
    
    def update_with_prices(self, price_data: List[PriceData]):
        for item in price_data:
            self.sensory_memory.add_price(item)
            self.short_term_memory.add_price(item)

# --- New Function: Update Agent with DataFrame ---

def update_agent_with_dataframe(agent: LLMAgent, df: pd.DataFrame):
    """
    For each row in the DataFrame, creates:
      - A NewsItem (using 'text' for content and 'date' for news date)
      - A PriceData with all columns (maps 'close' as the main price for asset "BTC")
    
    The news items are added to both sensory and short-term memory, and the price data
    (including additional fields) are similarly added.
    """
    news_items = []
    price_items = []
    
    for _, row in df.iterrows():
        # Create news item
        news_item = NewsItem(
            content=row["text"],
            date=row["date"],
            source="Crypto News",  # Default; adjust as needed
            category="Crypto"      # Default category; adjust as needed
        )
        # Create price data item with all available fields
        price_item = PriceData(
            asset="BTC",  # Assuming the asset is Bitcoin
            price=float(row["close"]),
            date=row["date"],
            open=float(row["open"]) if pd.notna(row["open"]) else None,
            high=float(row["high"]) if pd.notna(row["high"]) else None,
            low=float(row["low"]) if pd.notna(row["low"]) else None,
            volume=float(row["volume"]) if pd.notna(row["volume"]) else None,
            next_t_close=float(row["next_t_close"]) if pd.notna(row["next_t_close"]) else None,
            close_7=float(row["close_7"]) if pd.notna(row["close_7"]) else None,
            close_30=float(row["close_30"]) if pd.notna(row["close_30"]) else None,
            close_90=float(row["close_90"]) if pd.notna(row["close_90"]) else None,
            target=str(row["target"]) if pd.notna(row["target"]) else None,
            diff_perc=float(row["diff_perc"]) if pd.notna(row["diff_perc"]) else None
        )
        news_items.append(news_item)
        price_items.append(price_item)
    
    agent.update_with_news(news_items)
    agent.update_with_prices(price_items)

# --- Reward Function ---

def compute_reward(target: str, diff_perc: float) -> float:
    """
    Computes reward based on the actual outcome.
    The higher the diff_perc, the lower the reward.
    For example, we use: reward = max(0, 1 - diff_perc/10)
    """
    return max(0.0, 1 - diff_perc / 10)

# --- Simulation Example ---

if __name__ == "__main__":
    # Create a sample DataFrame from your updated data.
    data = {
        "date": [
            "2019-01-03 17:00:00", 
            "2019-01-03 17:00:00", 
            "2019-01-03 18:00:00", 
            "2019-01-03 18:00:00", 
            "2019-01-03 19:00:00"
        ],
        "text": [
            "Title:\n Bitcoin Blockchain – Experts Commen...",
            "Title:\n Overstock Will Pay Some of Its 2019 T...",
            "Title:\n How did cryptocurrency fare in 2018? ...",
            "Title:\n Trader: Bitcoin May See Long-Lasting ...",
            "Title:\n BTC Genesis Block – Major Milestone..."
        ],
        "timestamp": [
            "2019-01-03 17:00:00", 
            "2019-01-03 17:00:00", 
            "2019-01-03 18:00:00", 
            "2019-01-03 18:00:00", 
            "2019-01-03 19:00:00"
        ],
        "open": [3741.61, 3741.61, 3751.29, 3751.29, 3758.70],
        "high": [3761.94, 3761.94, 3759.56, 3759.56, 3764.60],
        "low": [3735.22, 3735.22, 3741.54, 3741.54, 3746.75],
        "close": [3751.29, 3751.29, 3758.70, 3758.70, 3757.09],
        "volume": [7609.91, 7609.91, 5130.42, 5130.42, 4018.77],
        "next_t_close": [3981.20, 3981.20, 3981.20, 3979.35, 3991.13],
        "close_7": [3756.50, 3748.524286, 3750.965714, 3753.407143, 3754.235714],
        "close_30": [3798.495000, 3796.037333, 3793.662333, 3792.025333, 3790.334667],
        "close_90": [3789.307000, 3789.958556, 3790.692444, 3791.432000, 3792.153667],
        "target": ["Long", "Long", "Long", "Long", "Long"],
        "diff_perc": [6.128825, 6.128825, 5.919600, 5.870381, 6.229289]
    }
    df = pd.DataFrame(data)
    
    # Initialize the agent.
    agent = LLMAgent()
    
    # Update the agent with the DataFrame.
    update_agent_with_dataframe(agent, df)
    
    # Make a recommendation based on the updated data.
    print("MAKING RECOMMENDATION BASED ON THE PROVIDED DATAFRAME")
    decision = agent.make_recommendation("Should investors consider increasing their allocation to Bitcoin?")
    
    print(f"\nRECOMMENDATION: {decision.recommendation}")
    print(f"CONFIDENCE: {decision.confidence}")
    print(f"REASONING: {decision.reasoning}")
    print(f"DECISION ID: {decision.decision_id}")
    
    # --- Simulate Feedback ---
    # For demonstration, we pick the last row from the DataFrame as the actual outcome.
    outcome_row = df.iloc[-1]
    actual_target = outcome_row["target"]
    diff_perc = float(outcome_row["diff_perc"])
    reward = compute_reward(actual_target, diff_perc)
    
    # Use outcome details (here we include the target and diff percentage in the outcome string)
    outcome_description = f"Actual outcome was {actual_target} with a diff_perc of {diff_perc:.2f}."
    
    print("\nPROCESSING FEEDBACK BASED ON ACTUAL OUTCOME")
    learning_output = agent.process_feedback(decision_id=decision.decision_id, outcome=outcome_description, reward=reward)
    
    print(f"\nLEARNING OUTPUT: {learning_output}")
    
    # (Optional) Visualize the agent's decision history.
    decisions = agent.autobiographical_memory.decisions
    if decisions:
        plt.figure(figsize=(10, 6))
        decision_ids = [d.decision_id[-6:] for d in decisions]  # Shortened IDs for display
        confidence_values = [d.confidence for d in decisions]
        rewards = [d.reward if d.reward is not None else 0 for d in decisions]
        
        bar_width = 0.35
        index = range(len(decisions))
        
        plt.bar([i - bar_width/2 for i in index], confidence_values, bar_width, label='Confidence')
        plt.bar([i + bar_width/2 for i in index], rewards, bar_width, label='Reward')
        
        plt.xlabel('Decisions')
        plt.ylabel('Value')
        plt.title('Agent Decision Confidence vs Rewards')
        plt.xticks(index, decision_ids)
        plt.legend()
        
        plt.tight_layout()
        plt.show()
